This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with only gain corruptions and a gain-only solver,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains with a user-defined sky model
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, Fint200=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, Fint200=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-16 11:35:53,613 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-16 11:35:53,614 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-16 11:35:53,615 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-16 11:35:53,628 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-16 11:35:54,173 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-16 11:35:54,330 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-16 11:35:54,899 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-16 11:35:54,900 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-16 11:35:55,662 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-16 11:35:55,831 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12

pipeline ran in 4.6 seconds


In [4]:
# Simple bandpass calibration of X and Y gains with GLEAM and EveryBeam
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-16 11:35:58,268 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-16 11:35:58,269 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2024-12-16 11:35:58,269 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-16 11:35:58,270 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2024-12-16 11:35:58,271 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-16 11:35:58,272 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2024-12-16 11:35:58,272 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2024-12-16 11:35:58,874 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-16 11:35:58,875 - workflow.pipeline_config - INFO - LSM: found 37 components
2024-12-16 11:35:58,898 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-16 11:35:58,937 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-16 11:36:02,267 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-16 11:36:02,828 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-16 11:36:02,829 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-16 11:36:03,579 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-16 11:36:03,651 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2024-12-16 11:36:03,652 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-16 11:36:03,653 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2024-12-16 11:36:03,654 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2024-12-16 11:36:04,281 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-16 11:36:04,282 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2024-12-16 

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-16 11:36:08,134 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-16 11:36:08,155 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-16 11:36:08,159 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 10.3 seconds


In [5]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-16 11:36:08,564 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-16 11:36:08,565 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2024-12-16 11:36:08,565 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-16 11:36:08,566 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2024-12-16 11:36:08,566 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-16 11:36:08,567 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2024-12-16 11:36:08,567 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2024-12-16 11:36:09,156 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2024-12-16 11:36:09,157 - workflow.pipeline_config - INFO - LSM: found 37 components
2024-12-16 11:36:09,167 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-16 11:36:09,198 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-16 11:36:12,417 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-16 11:36:12,420 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2024-12-16 11:36:12,979 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-16 11:36:12,980 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-16 11:36:13,727 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-16 11:36:13,789 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2024-12-16 11:36:13,790 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2024-12-16 11:36:13,791 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2024-12-16 11:36:13,792 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2024-12-16 11:36:14,500 - processing_tasks.lsm - INFO - extracted 37 GLEAM components


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-16 11:36:19,460 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-16 11:36:19,474 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-16 11:36:19,476 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 11.3 seconds
